### **Function:** *`_fit_circle_lstsq`*  
This function implements a mathematical **circle fitting algorithm** using a **linear least-squares approximation**.  
It takes a set of 2D points as input and solves a system of linear equations to determine the parameters of a circle:  

\[
x^2 + y^2 - 2gx - 2fy - c = 0
\]

The function returns the calculated **center coordinates** \((c_x, c_y)\) and the **radius (r)** of the best-fit circle for the provided points.

---

### **Function:** *`robust_fit_circle`*  
This function provides a **robust method** for determining the center of a circle from a set of **noisy 2D points**.  
It enhances the standard least-squares fit by incorporating an **outlier rejection mechanism**.  
The methodology involves:
1. An initial fit to establish a baseline.  
2. Identification and exclusion of outlier points using the **Median Absolute Deviation (MAD)** method.  
3. A final, more accurate fit on the inlier set to yield a refined center position that is **resilient to noise**.

---

### **Function:** *`find_bubble_by_inward_scan`*  
This function serves as the **primary detection algorithm** for identifying the **bubble boundary**.  
It operates on the image’s **gradient magnitude**, computed using a **Sobel operator**, which highlights sharp intensity transitions (edges).  

The algorithm uses a **radial scanning technique**:
- 180 rays are cast from an initial center point.  
- The algorithm traverses **inwards** along each ray, identifying the first pixel exceeding a gradient threshold.  
- The collected edge points are then filtered using **robust statistics (MAD)** to remove outliers.  

Finally, it returns the **mean radius** of the remaining inlier edge points.

---

### **Function:** *`find_bubble_with_iterative_refinement`*  
This function orchestrates the detection process for **high-precision measurements**.  
It employs an **iterative refinement strategy** to locate the bubble’s true center and radius.  

Steps:
1. Begin with an initial detection using the geometric center of the frame.  
2. Compute edge points and calculate a refined center via `robust_fit_circle`.  
3. Use this refined center for the next iteration.  
4. Track the **stability (standard deviation)** of the radii across iterations.  

The function returns the result from the **most consistent (reliable)** iteration.

---

### **Function:** *`analyze_bubble_dynamics`*  
This is the **main driver function** that manages the entire analysis pipeline.  
It processes each of the four **bubble sequences** defined in the configuration.  

For each frame:
- Performs precise cropping using the manually specified grid coordinates.  
- Invokes `find_bubble_with_iterative_refinement` for accurate radius measurement.  
- Generates a visualization where the detected bubble boundary is overlaid as a **green circle**.  
- Outputs quantitative results to the console and plots a **Radius vs. Time** graph for each sequence.


In [ ]:
# Step 1: Upload Image File
from google.colab import files
import os

if 'uploaded_file_name' in locals():
    del uploaded_file_name

uploaded = files.upload()

if uploaded:
    uploaded_file_name = next(iter(uploaded))
    print(f"\nFile '{uploaded_file_name}' uploaded successfully.")
else:
    print("\nFile upload cancelled or failed.")

In [ ]:
#Step 2: Run Bubble Analysis

import cv2
import numpy as np
import matplotlib.pyplot as plt
import math

if 'uploaded_file_name' not in locals() or not uploaded_file_name:
    print("Execution halted: Please run the first cell to upload an image.")
else:
    # --- 1. SCRIPT CONFIGURATION ---
    IMAGE_PATH = uploaded_file_name

    # Manually defined grid boundaries for precise frame cropping.
    GRID_START_X_PIXELS = 30
    MANUAL_ROW_BOUNDARIES_Y = [(25, 150), (180, 310), (340, 500), (530, 760)]

    # Time values (in ms) corresponding to each frame in each sequence.
    ALL_MANUAL_TIME_VALUES = [
        [0, 0.52, 0.84, 1.16, 1.60, 2.04, 2.48, 2.72],
        [0, 1.36, 1.68, 1.88, 2.36, 2.84, 3.32, 3.44],
        [0, 1.32, 1.80, 2.12, 2.52, 2.92, 3.32, 3.40],
        [0, 1.04, 1.36, 1.88, 2.36, 2.84, 3.28, 3.36]
    ]

    # --- 2. ALGORITHM IMPLEMENTATION ---

    def _fit_circle_lstsq(points):
        # Implements a circle fit using a linear least-squares method.
        if points is None or len(points) < 3: return None, None
        x, y = points[:, 0], points[:, 1]
        A = np.vstack([x, y, np.ones(len(x))]).T
        B = x**2 + y**2
        try:
            c, _, _, _ = np.linalg.lstsq(A, B, rcond=None)
        except np.linalg.LinAlgError:
            return None, None
        cx, cy = c[0] / 2, c[1] / 2
        r = np.sqrt(c[2] + cx**2 + cy**2)
        return (cx, cy), r

    def robust_fit_circle(points):
        # Refines the circle's center by rejecting outliers with Median Absolute Deviation.
        if points is None or len(points) < 10: return None
        initial_center, initial_radius = _fit_circle_lstsq(points)
        if initial_center is None: return None

        distances = np.sqrt((points[:, 0] - initial_center[0])**2 + (points[:, 1] - initial_center[1])**2)
        residuals = np.abs(distances - initial_radius)
        median_residual = np.median(residuals)
        mad_residual = np.median(np.abs(residuals - median_residual))

        if mad_residual < 1e-6:
            return (int(round(initial_center[0])), int(round(initial_center[1])))

        inlier_mask = residuals < (median_residual + 3.0 * mad_residual)
        if np.sum(inlier_mask) < 10:
            return (int(round(initial_center[0])), int(round(initial_center[1])))

        final_center, _ = _fit_circle_lstsq(points[inlier_mask])
        if final_center is None:
            return (int(round(initial_center[0])), int(round(initial_center[1])))
        return (int(round(final_center[0])), int(round(final_center[1])))

    def find_bubble_by_inward_scan(sub_image, center_coords=None):
        # Detects the bubble boundary by performing a radial scan on the image gradient.
        gray = cv2.cvtColor(sub_image, cv2.COLOR_BGR2GRAY)
        blurred = cv2.GaussianBlur(gray, (7, 7), 0)

        grad_x = cv2.Sobel(blurred, cv2.CV_64F, 1, 0, ksize=5)
        grad_y = cv2.Sobel(blurred, cv2.CV_64F, 0, 1, ksize=5)
        grad_mag = cv2.magnitude(grad_x, grad_y)

        h, w = sub_image.shape[:2]
        non_zero_grads = grad_mag[grad_mag > 5]
        if len(non_zero_grads) < 100: return 0.0, None

        grad_threshold = np.median(non_zero_grads) + 1.5 * np.std(non_zero_grads)
        grad_threshold = max(grad_threshold, 40.0)

        center_x, center_y = center_coords if center_coords is not None else (w // 2, h // 2)
        num_rays, edge_points = 180, []

        for i in range(num_rays):
            angle = 2 * np.pi * i / num_rays
            max_r = int(min(w - center_x - 1, center_x, h - center_y - 1, center_y))
            if max_r < 10: continue
            for r in range(max_r, 10, -1):
                x, y = int(center_x + r * np.cos(angle)), int(center_y + r * np.sin(angle))
                if not (0 <= y < h and 0 <= x < w): continue
                if grad_mag[y, x] > grad_threshold:
                    edge_points.append((x, y, r)); break

        if len(edge_points) < num_rays * 0.2: return 0.0, None

        radii_np = np.array([p[2] for p in edge_points])
        median = np.median(radii_np)
        mad = np.median(np.abs(radii_np - median))
        if mad < 1e-6: mad = np.std(radii_np) * 1.4826

        inliers_mask = (np.abs(radii_np - median) < 3.0 * mad) if mad > 1e-6 else np.ones_like(radii_np, dtype=bool)
        if np.sum(inliers_mask) < 20: return 0.0, None

        final_points = np.array(edge_points)[inliers_mask]
        final_radius = np.mean(final_points[:, 2])
        return final_radius, final_points

    def find_bubble_with_iterative_refinement(sub_image):
        # Orchestrates the detection by iteratively refining the bubble's center.
        h, w = sub_image.shape[:2]
        current_center = (w // 2, h // 2)
        best_result = {'radius': 0.0, 'center': current_center, 'std_dev': float('inf')}

        for i in range(4):
            radius, points_data = find_bubble_by_inward_scan(sub_image, center_coords=current_center)
            if points_data is None: break

            std_dev_this_pass = np.std(points_data[:, 2])
            if std_dev_this_pass < best_result['std_dev']:
                best_result = {
                    'radius': np.mean(points_data[:, 2]),
                    'center': current_center,
                    'std_dev': std_dev_this_pass
                }

            new_center = robust_fit_circle(points_data[:, :2].astype(np.float32))
            if new_center is None: break

            dist_sq = (new_center[0] - current_center[0])**2 + (new_center[1] - current_center[1])**2
            if dist_sq < 1.0: break

            current_center = new_center

        return best_result['radius'], best_result['center']

    # --- 3. MAIN ANALYSIS DRIVER ---
    def analyze_bubble_dynamics(image_path, grid_start_x, row_boundaries, all_times):
        # Main function to control the analysis workflow, visualization, and output.
        try:
            full_image = cv2.imread(image_path)
            if full_image is None: raise FileNotFoundError(f"Image not found at {image_path}")
            output_image = full_image.copy()
        except Exception as e:
            print(f"Error loading image: {e}"); return

        _, img_width, _ = full_image.shape
        cell_width_calc = (img_width - grid_start_x) // 8
        all_results = []

        print("\nInitiating bubble dynamics analysis...")
        for row_index, time_values in enumerate(all_times):
            calculated_radii = []
            y1, y2 = row_boundaries[row_index]

            for col_index, time in enumerate(time_values):
                x1 = grid_start_x + (col_index * cell_width_calc)

                if time == 0:
                    calculated_radii.append(0.0); continue

                sub_image = full_image[y1:y2, x1:(x1 + cell_width_calc)]

                radius, center = find_bubble_with_iterative_refinement(sub_image)
                calculated_radii.append(radius)

                # Overlay the detected circle on the visualization image.
                if radius > 0 and center is not None:
                    center_global = (x1 + center[0], y1 + center[1])
                    cv2.circle(output_image, center_global, int(radius), (0, 255, 0), 2)

            all_results.append({'times': time_values, 'radii': calculated_radii})

        print("Analysis complete.")

        # Display the composite visualization image.
        plt.figure(figsize=(20, 10))
        plt.imshow(cv2.cvtColor(output_image, cv2.COLOR_BGR2RGB))
        plt.title("Bubble Detection Visualization", fontsize=16)
        plt.axis('off')
        plt.show()

        # Print and plot the final results for each sequence.
        for i, result in enumerate(all_results):
            print(f"\n--- Results for Bubble Sequence {i + 1} ---")
            for t, r in zip(result['times'], result['radii']):
                print(f"Time: {t:<5.2f} ms,   Radius: {r:<7.2f} pixels")

            plt.figure(figsize=(10, 6))
            plt.plot(result['times'], result['radii'], marker='o', linestyle='-')
            plt.title(f"Bubble Sequence {i + 1}: Radius vs. Time")
            plt.xlabel("Time (ms)"); plt.ylabel("Radius (pixels)")
            plt.grid(True); plt.ylim(bottom=0); plt.xlim(left=0)
            plt.show()

        return all_results, full_image

    # --- 4. EXECUTION: Run analysis and capture results ---
all_results, full_image = analyze_bubble_dynamics(
    IMAGE_PATH, GRID_START_X_PIXELS, MANUAL_ROW_BOUNDARIES_Y, ALL_MANUAL_TIME_VALUES
)


In [ ]:
# ==============================
# Step 3: Generate Premium Bubble Animations
# ==============================
import cv2
import numpy as np
import imageio
import os

def remove_electrode(sub_img):
    """
    Removes bright electrode regions from the sub-image to make animation clean.
    """
    gray = cv2.cvtColor(sub_img, cv2.COLOR_BGR2GRAY)
    h, w = gray.shape
    col_mean = gray.mean(axis=0)
    thr = np.median(col_mean) + 0.6 * np.std(col_mean)
    bright_cols = np.where(col_mean > thr)[0]

    mask = np.zeros_like(gray, dtype=np.uint8)
    if len(bright_cols) > 0:
        left = max(0, bright_cols[0] - 5)
        right = min(w - 1, bright_cols[-1] + 5)
        mask[:, left:right] = 255
    else:
        mask[:, :int(w*0.15)] = 255

    cleaned = cv2.inpaint(sub_img, mask, 5, cv2.INPAINT_TELEA)
    return cleaned

def draw_bubble(canvas, center, radius, color=(0, 0, 255)):
    img = canvas.copy()
    if radius > 0:
        cv2.circle(img, (int(center[0]), int(center[1])), int(radius), color, 2)
    return img

def interpolate_bubbles(frame1, c1, r1, frame2, c2, r2, steps=10):
    """
    Simulates smooth bubble propagation between two frames.
    """
    frames = []
    for s in range(1, steps+1):
        t = s / steps
        blended = cv2.addWeighted(frame1, 1-t, frame2, t, 0)
        cx = int(round(c1[0]*(1-t) + c2[0]*t))
        cy = int(round(c1[1]*(1-t) + c2[1]*t))
        r = r1*(1-t) + r2*t
        frames.append(draw_bubble(blended, (cx, cy), r))
    return frames

def generate_premium_bubble_animation(all_results, full_image, row_boundaries, output_dir='bubble_animations_premium'):
    os.makedirs(output_dir, exist_ok=True)
    print("\n--- Generating premium bubble animations ---")

    H, W, _ = full_image.shape

    for seq_idx, seq in enumerate(all_results):
        y1, y2 = row_boundaries[seq_idx]
        canvas_height = y2 - y1
        canvas_width = W - GRID_START_X_PIXELS
        base_canvas = 255 * np.ones((canvas_height, canvas_width, 3), dtype=np.uint8)
        frames = []

        # Convert all bubbles into coordinates on canvas
        bubble_coords = []
        num_frames = len(seq['radii'])
        for col_idx, radius in enumerate(seq['radii']):
            if radius == 0: continue
            cx = col_idx * (canvas_width // num_frames) + (canvas_width // (2*num_frames))
            cy = canvas_height // 2
            bubble_coords.append((cx, cy, radius))

        # Generate frames with interpolation
        for i in range(len(bubble_coords)-1):
            f1 = draw_bubble(base_canvas, bubble_coords[i][:2], bubble_coords[i][2])
            f2 = draw_bubble(base_canvas, bubble_coords[i+1][:2], bubble_coords[i+1][2])
            frames.append(f1)
            interp_frames = interpolate_bubbles(f1, bubble_coords[i][:2], bubble_coords[i][2],
                                                f2, bubble_coords[i+1][:2], bubble_coords[i+1][2],
                                                steps=8)  # 8 interpolated frames
            frames.extend(interp_frames)
        frames.append(draw_bubble(base_canvas, bubble_coords[-1][:2], bubble_coords[-1][2]))

        # Save frames as video and GIF
        frames_rgb = [cv2.cvtColor(f, cv2.COLOR_BGR2RGB) for f in frames]
        out_mp4 = os.path.join(output_dir, f"bubble_sequence_{seq_idx+1}.mp4")
        out_gif = os.path.join(output_dir, f"bubble_sequence_{seq_idx+1}.gif")
        imageio.mimsave(out_mp4, frames_rgb, fps=15)
        imageio.mimsave(out_gif, frames_rgb, fps=15)
        print(f"Saved: {out_mp4} and {out_gif}")

    print("\nPremium bubble animations generated!")

# --- Run premium animation using Step 2 results ---
generate_premium_bubble_animation(all_results, full_image, MANUAL_ROW_BOUNDARIES_Y)



--- Generating premium bubble animations ---


Saved: bubble_animations_premium/bubble_sequence_1.mp4 and bubble_animations_premium/bubble_sequence_1.gif


Saved: bubble_animations_premium/bubble_sequence_2.mp4 and bubble_animations_premium/bubble_sequence_2.gif


Saved: bubble_animations_premium/bubble_sequence_3.mp4 and bubble_animations_premium/bubble_sequence_3.gif
Saved: bubble_animations_premium/bubble_sequence_4.mp4 and bubble_animations_premium/bubble_sequence_4.gif

✨ Premium bubble animations generated!
